In [1]:
import sys
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-client.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-biodatacatalyst-python-adapter-hpds.git

  Cloning https://github.com/hms-dbmi/pic-sure-python-client.git to /tmp/pip-req-build-0o41cgbq
  Running command git clone --filter=blob:none --quiet https://github.com/hms-dbmi/pic-sure-python-client.git /tmp/pip-req-build-0o41cgbq
  Resolved https://github.com/hms-dbmi/pic-sure-python-client.git to commit b1c5419290fd1d7ecc1494698caae5436fb6a2e8
  Preparing metadata (setup.py) ... done
  Created wheel for PicSureClient: filename=PicSureClient-0.1.0-py2.py3-none-any.whl size=10646 sha256=594db768c395baa926b5b8040181ad5da873096adab063f403221db38f815036
  Stored in directory: /tmp/pip-ephem-wheel-cache-j6z4cog3/wheels/90/65/c4/e74447484bdae71b64f3f0a500bc7b3d9d6ee7edc62ade6667
Successfully built PicSureClient
  Attempting uninstall: PicSureClient
    Found existing installation: PicSureClient 0.1.0
    Uninstalling PicSureClient-0.1.0:
      Successfully uninstalled PicSureClient-0.1.0
  Cloning https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git to /tmp/pip-req-build-93p19fck

In [3]:
import PicSureClient
import PicSureBdcAdapter

Load PIC-SURE API

In [27]:
# Integration:
#PICSURE_network_URL = "https://biodatacatalyst.integration.hms.harvard.edu/picsure"
# Production:
PICSURE_network_URL = "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
token_file = "token.txt"

with open(token_file, "r") as f:
    my_token = f.read()
    
bdc = PicSureBdcAdapter.Adapter(PICSURE_network_URL, my_token)

+--------------------------------------+------------------------------------------------------+
|  Resource UUID                       |  Resource Name                                       |
+--------------------------------------+------------------------------------------------------+
| 70c837be-5ffc-11eb-ae93-0242ac130002 | open-hpds                                            |
| ca0ad4a9-130a-3a8a-ae00-e35b07f1108b | visualization                                        |
| 02e23f52-f354-4e8b-992c-d37c8b9ba140 | auth-hpds                                            |
| 36363664-6231-6134-2d38-6538652d3131 | dictionary                                           |
+--------------------------------------+------------------------------------------------------+


In [28]:
dictionary = bdc.useDictionary().dictionary() # Set up the dictionary
all_vars = dictionary.find('phs000287') # Fill in with phs number of interest
all_variables = all_vars.dataframe() # Retrieve all variables you have access to

In [30]:
#all_variables#["derived_var_name"]

In [31]:
ids = all_variables["derived_var_name"]
pd.unique(ids).size

6497

In [32]:
dup_vars_series = ids.value_counts().loc[lambda x : x > 1]
dup_vars_df = pd.DataFrame({'id':dup_vars_series.index, 'count':dup_vars_series.values})

In [46]:
issues_df = pd.DataFrame()

for id in dup_vars_df['id']:
    #print(id)
    var_df = all_variables.loc[all_variables['derived_var_name'] == id]
    if var_df['is_categorical'].unique().size > 1 or var_df['is_continuous'].unique().size > 1 :
        #print(id)
        issues_df = pd.concat([issues_df, var_df[['derived_var_name','dtId','varId','is_categorical','is_continuous']]],ignore_index=True)
        
issues_df

,derived_var_name,dtId,varId,is_categorical,is_continuous
0,PERSTAT,pht001452,phv00100290,False,True
1,PERSTAT,pht001466,phv00100809,True,False
2,PERSTAT,pht001470,phv00101122,True,False
3,PERSTAT,pht001472,phv00101232,True,False
4,PERSTAT,pht001473,phv00101234,True,False
...,...,...,...,...,...
809,TTLINT59,pht001493,phv00108714,False,True
810,mxoanbp,pht003700,phv00199174,True,False
811,mxoanbp,pht003699,phv00197974,False,True
812,CAUSE80,pht001464,phv00100775,True,False


In [49]:
issues_df.to_csv("variable_inconsistencies.csv",index=False)